# GANomaly Notebook Results

## Initial Configurations

### Libraries import

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import umap
import cv2
from IPython.display import display
from ipywidgets import interact, IntSlider

sys.path.append("../../")

In [2]:
from utils.metrics import shapiroWilks_test, dagostinoPearson_test, bartlett_test, levene_test, fOneWay_test
from utils.savers import generate_qq_plot

## Quantitative metrics

### Metrics loading

In [3]:
train_metrics = pd.read_csv("/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/metrics/train.csv")
data = train_metrics
np.max(data, axis=0), np.min(data, axis=0), data.to_numpy()[-1]

(epoch              29999
  gen_error       164.954
  disc_error      7.62462
  accuracy       0.930556
  precision             0
  recall              nan
  specificity    0.930556
  f1_score              0
  auc                   0
 dtype: object,
 epoch                  0
  gen_error      0.530382
  disc_error      7.62462
  accuracy       0.416667
  precision             0
  recall              nan
  specificity    0.416667
  f1_score              0
  auc                   0
 dtype: object,
 array([29999, 0.5910592675209045, 7.624619007110597, 0.6111111044883728,
        0.0, ' nan', 0.6111111044883728, 0.0, 0.0], dtype=object))

In [4]:
test_metrics = pd.read_csv("/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/metrics/test.csv")
data = test_metrics
np.max(data, axis=0), np.min(data, axis=0), data.to_numpy()[-1]

(epoch           29999.000000
  accuracy           0.903846
  precision          1.000000
  recall             0.886364
  specificity        1.000000
  f1_score           0.939759
  auc                1.000000
 dtype: float64,
 epoch           0.000000
  accuracy       0.125000
  precision      0.363636
  recall         0.045455
  specificity    0.250000
  f1_score       0.080808
  auc            0.113991
 dtype: float64,
 array([2.99990000e+04, 3.84615391e-01, 1.00000000e+00, 2.72727281e-01,
        1.00000000e+00, 4.28571429e-01, 1.00000000e+00]))

In [ ]:
data = test_metrics
np.max(data, axis=0), np.min(data, axis=0), data.to_numpy()[-1]

### Train Graphics

In [ ]:
path = '/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/graphics/quantitative/'
portions = 500
rows = 12
columns = 5
for metric in [" gen_error", " disc_error", " accuracy", " specificity"]:
    fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
    fig.suptitle('{} over Epochs'.format(metric[1:]))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(train_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'train_{}.png'.format(metric[1:])
    fig.savefig(path+filename)
    plt.close(fig)

### Test graphics

In [ ]:
path = '/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/graphics/quantitative/'
portions = 500
rows = 12
columns = 5
for metric in test_metrics.columns[1:]:
    fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
    fig.suptitle('{} over Epochs'.format(metric[1:]))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(test_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'test_{}.png'.format(metric[1:])
    fig.savefig(path+filename)
    plt.close(fig)

## Qualitative metrics

### Errors loading

In [ ]:
base_path = '/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/errors/'

for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            globals()["{}_{}_{}".format(m, t, c)] = np.load(
                os.path.join(base_path, t, m, c+".npy")
            )

### Train means and stds

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        data = globals()["train_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

### Train statistical tests

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        print("-------------------------------------------------")
        data = globals()["train_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

### Test means and stds

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["test_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

### Test statistical tests

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["test_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

### Train graphics

#### QQ plots

In [ ]:
save_path = '/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/graphics/qualitative/'
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        generate_qq_plot(globals()["train_{}_{}".format(t,c)], save_path, "train_{}_{}".format(t,c))

### Test graphics

#### Box plots

In [ ]:
save_path = '/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/graphics/qualitative/'
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "test_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'test_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [ ]:
save_path = '/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/graphics/qualitative/'
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["test_{}_{}".format(t,c)], save_path, "test_{}_{}".format(t,c))

## Visual results

### UMAP latent vectors

In [ ]:
base_path = '/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/latent_vectors/'

for t in ["in", "out"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            data = "{}_{}_{}".format(m, t, c)
            globals()[data] = []
            path = os.path.join(base_path, t, m, c)
            for file in sorted(os.listdir(path)):
                globals()[data].append(
                    np.load(os.path.join(path, file))
                )
            globals()[data] = np.r_[globals()[data]]

In [ ]:
save_path = "/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/graphics/visuals/"
for t in ["in", "out"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        globals()["{}_mapped_{}".format(m, t)] = umap.UMAP(random_state = 8128).fit_transform(np.concatenate(
                [globals()["{}_{}_{}".format(m, t, c)] for c in classes], axis=0
            )
        )
        
        divisor = globals()["{}_{}_normal".format(m, t)].shape[0]
        plt.scatter(globals()["{}_mapped_{}".format(m, t)][:divisor,0],
            globals()["{}_mapped_{}".format(m, t)][:divisor,1], color="blue", s=0.5, label="Normal"
        )
        if "abnormal" in classes:
            plt.scatter(globals()["{}_mapped_{}".format(m, t)][divisor:,0],
                globals()["{}_mapped_{}".format(m, t)][divisor:,1], color="red", s=0.5, label="Abnormal"
            )
        plt.legend()
        plt.title("UMAP for {}put data in {}".format(t, m))
        filename = '{}_umap_{}put.png'.format(m, t)
        plt.savefig(save_path+filename)
        plt.close()

### Video comparision

In [7]:
video_index = 2
mode = "test"
video_class = "abnormal"

base_path = "/home/jefelitman/Saved_Models/Anomaly_parkinson/0005_Ganomaly_3D-64x64x64x3/outputs/samples/"
for t in ["real", "fake", "substraction"]:
    path = os.path.join(base_path, t, mode, video_class)
    video = sorted(os.listdir(path))[video_index]
    data = "video_{}".format(t)
    globals()[data] = []
    for frame in sorted(os.listdir(os.path.join(path, video))):
        globals()[data].append(cv2.cvtColor(
            cv2.imread(
                os.path.join(path, video, frame)
            ), cv2.COLOR_BGR2RGB
        ))
    globals()[data] = np.r_[globals()[data]]

In [8]:
assert video_real.shape[0] == video_fake.shape[0] == video_substraction.shape[0]

widget = IntSlider(min=1, max=video_real.shape[0], step=1)

interact(lambda frame: plt.imshow(video_real[frame-1]), frame=widget)
interact(lambda frame: plt.imshow(video_fake[frame-1]), frame=widget)
interact(lambda frame: plt.imshow(video_substraction[frame-1]), frame=widget)

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

<function __main__.<lambda>(frame)>